# Summary statistics + exploratory data analysis
report:
- sample size (by mode/dataset, if needed)
- mapping of distribution of origin/destination of trips
- time of day distribution
- trip distance/travel time distribution
- breakdown by mode

## some research into mining frequent itemsets using python packages
Algorithm: Apriori (or FP growth?)

- MLXtend
- itemset-mining
- apyori
- PyCaret
- MLlib (FP-growth)-- this is what we used in the colab
